# Lambda phage

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pysb.integrate import odesolve
from scripts.models.lambda_phage_pysb import model
import xarray as xr

from scripts.grid_class import GridParms
from scripts.tree_class import Tree
from scripts.output.output_helper import *
from scripts.reference_solutions.ssa_helper import SSASol

plt.style.use("./scripts/output/notebooks/custom_style.mplstyle")
%matplotlib inline

In [ ]:
concentrations_ode = odesolve(model, np.arange(10.0))
slice_vec = []
for o in model.observables:
    slice_vec.append(int(np.round(concentrations_ode[o.name][-1])))
slice_vec = np.array(slice_vec)

## Load initial data

In [ ]:
with np.load("scripts/reference_solutions/lp_ode_ref.npz") as data:
    P_exact = data['P_full']
    P_best_approximation = data['P_best_approximation']

In [ ]:
ssa_1e4 = np.load("scripts/reference_solutions/lp_ssa_1e4.npy")
ssa_1e4_sol = SSASol(ssa_1e4)
P_ssa_1e4 = ssa_1e4_sol.calculateFullDistribution()

In [ ]:
ssa_1e5 = np.load("scripts/reference_solutions/lp_ssa_1e5.npy")
ssa_1e5_sol = SSASol(ssa_1e5)
P_ssa_1e5 = ssa_1e5_sol.calculateFullDistribution()

In [ ]:
# ssa_1e6 = np.load("scripts/reference_solutions/lp_ssa_1e6.npy")
# ssa_1e6_sol = SSASol(ssa_1e6)
# P_ssa_1e6 = ssa_1e6_sol.calculateFullDistribution()

## Comparison of TTN and SSA results with matrix integrator

### Load data

#### Get walltimes

In [ ]:
time_series = TimeSeries("output/lambda_phage_p0_r5-3_e_tau1e-3")
walltime_p0_r_5_3 = time_series.getWallTime()

In [ ]:
time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau1e-3")
walltime_p0_r_5_4 = time_series.getWallTime()

In [ ]:
time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-3")
walltime_p0_r_5_5 = time_series.getWallTime()

In [ ]:
time_series = TimeSeries("output/lambda_phage_p0_r5-6_e_tau1e-3")
walltime_p0_r_5_10 = time_series.getWallTime()

In [ ]:
time_series = TimeSeries("output/lambda_phage_pfull_r5_e_tau1e-3")
walltime_pfull = time_series.getWallTime()

In [ ]:
walltime_exact = 1164

In [ ]:
walltime_ssa_1e4 = 855
walltime_ssa_1e5 = 905
# walltime_ssa_1e6 = 2319

In [ ]:
walltimes = [walltime_pfull, walltime_p0_r_5_10, walltime_p0_r_5_5, walltime_p0_r_5_3, walltime_exact]
walltimes_ssa = [walltime_ssa_1e4, walltime_ssa_1e5, walltime_exact]
labels = ["2 partitions", "$r = (5, 6)$", "$r = (5, 5)$", "$r = (5, 3)$", "exact"]
labels_ssa = ["$10^4$ runs", "$10^5$ runs", "exact"]

#### Error between TTN integrator and the matrix integrator reference solution

In [ ]:
time_series = TimeSeries("output/lambda_phage_p0_r5-3_e_tau1e-3")
time = time_series.time
P_p0_r5_3_tau_1e3 = time_series.calculateFullDistribution()
err_r_5_3 = np.linalg.norm(P_p0_r5_3_tau_1e3 - P_exact, axis=1, ord=np.inf)

time_series = TimeSeries("output/lambda_phage_p0_r5-4_e_tau1e-3")
P_p0_r5_4_tau_1e3 = time_series.calculateFullDistribution()
err_r_5_4 = np.linalg.norm(P_p0_r5_4_tau_1e3 - P_exact, axis=1, ord=np.inf)

time_series = TimeSeries("output/lambda_phage_p0_r5-5_e_tau1e-3")
P_p0_r5_5_tau_1e3 = time_series.calculateFullDistribution()
err_r_5_5 = np.linalg.norm(P_p0_r5_5_tau_1e3 - P_exact, axis=1, ord=np.inf)

time_series = TimeSeries("output/lambda_phage_p0_r5-6_e_tau1e-3")
P_p0_r5_10_tau_1e3 = time_series.calculateFullDistribution()
err_r_5_10 = np.linalg.norm(P_p0_r5_10_tau_1e3 - P_exact, axis=1, ord=np.inf)

time_series = TimeSeries("output/lambda_phage_pfull_r5_e_tau1e-3")
P_pfull_r5_tau_1e3 = time_series.calculateFullDistribution()
err_r_5 = np.linalg.norm(P_pfull_r5_tau_1e3 - P_exact, axis=1, ord=np.inf)

In [ ]:
err_ssa_1e4 = np.linalg.norm(P_ssa_1e4 - P_exact, axis=1, ord=np.inf)
err_ssa_1e5 = np.linalg.norm(P_ssa_1e5 - P_exact, axis=1, ord=np.inf)

In [ ]:
err_best_approximation = np.linalg.norm(P_best_approximation - P_exact, axis=1, ord=np.inf)

### Plots

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(5, 7), layout="constrained")

general_labels = ["2 partitions", "best-approximation"]
axs[0].plot(time[1:], err_r_5_3[1:], '.-', label="$r = (5, 3)$")
axs[0].plot(time[1:], err_r_5_4[1:], '.-', label="$r = (5, 4)$")
axs[0].plot(time[1:], err_r_5_5[1:], '.-', label="$r = (5, 5)$")
axs[0].plot(time[1:], err_r_5_10[1:], '.-', label="$r = (5, 6)$")
l1, = axs[0].plot(time[1:], err_r_5[1:], 'k:')
l2, = axs[0].plot(time[1:], err_best_approximation[1:], 'k-.')

axs[0].set_yscale("log")
axs[0].set_xlabel("$t$")
axs[0].legend()
axs[0].set_ylabel("$\max_{{x}}|P^\mathrm{{TTN}}(t,x)-P^\mathrm{{ref}}(t,x)|$")

axs[1].barh(labels, walltimes)
axs[1].set_xscale("log")
axs[1].set_xlabel("wall time [$\mathrm{{s}}$]")
plt.subplots_adjust(wspace=.3, hspace=.3)
fig.legend([l1, l2], general_labels, loc="outside upper center", ncols=2)

plt.savefig("plots/comparison_marginal_ttn_ssa.pdf");

## Error depending on time step size

In [ ]:
dm_max_r_5_10 = []
dm_max_r_5_5 = []
dm_max_r_5_3 = []
tau = []

time_series = TimeSeries("output/lambda_phage_p0_r5-10_i_tau5e-1")
dm_max_r_5_10.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-5_i_tau5e-1")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-3_i_tau5e-1")
dm_max_r_5_3.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

time_series = TimeSeries("output/lambda_phage_p0_r5-10_i_tau2e-1")
dm_max_r_5_10.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-5_i_tau2e-1")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-3_i_tau2e-1")
dm_max_r_5_3.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

time_series = TimeSeries("output/lambda_phage_p0_r5-10_i_tau1e-1")
dm_max_r_5_10.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-5_i_tau1e-1")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-3_i_tau1e-1")
dm_max_r_5_3.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

time_series = TimeSeries("output/lambda_phage_p0_r5-10_i_tau5e-2")
dm_max_r_5_10.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-5_i_tau5e-2")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-3_i_tau5e-2")
dm_max_r_5_3.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

time_series = TimeSeries("output/lambda_phage_p0_r5-10_i_tau2e-2")
dm_max_r_5_10.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-5_i_tau2e-2")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-3_i_tau2e-2")
dm_max_r_5_3.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

time_series = TimeSeries("output/lambda_phage_p0_r5-10_i_tau1e-2")
dm_max_r_5_10.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-5_i_tau1e-2")
dm_max_r_5_5.append(np.abs(time_series.getMaxMassErr()))
time_series = TimeSeries("output/lambda_phage_p0_r5-3_i_tau1e-2")
dm_max_r_5_3.append(np.abs(time_series.getMaxMassErr()))
tau.append(time_series.getTau())

### Plot

In [ ]:
fig, ax = plt.subplots()
t = np.linspace(0.01, 0.5)

ax.loglog(tau, dm_max_r_5_3, '.-', label="$r=(5, 3)$")
ax.loglog(tau, dm_max_r_5_5, '.-', label="$r=(5, 5)$")
ax.loglog(tau, dm_max_r_5_10, '.-', label="$r=(5, 10)$")
ax.loglog(t, t**2*0.1, 'k:', label="$\\tau^2$")
ax.set_xlabel("time step size $\\tau$")
ax.set_ylabel("$\max(\Delta m)$")
fig.legend(loc="upper center", ncols=5)

plt.savefig("plots/time_mass_err_ttn.pdf");

## Comparison with deterministic solution

In [ ]:
time_series = TimeSeries("output/lambda_phage_p0_r5-5_i_tau5e-2")
concentrations = time_series.calculateMoments()
t = time_series.time

In [ ]:
concentrations_ode = odesolve(model, t)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 4))
deviation = np.sqrt(concentrations[1]-concentrations[0]**2)
observables = ["x1", "x2"]
observables_alt = ["$S_0$", "$S_1$", "$S_2$"]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
for i, (o, o_alt) in enumerate(zip(observables, observables_alt)):
    ax1.plot(t, concentrations[0][:, i], '-', label=o_alt, color=colors[i], alpha=0.7)
    ax1.fill_between(t, concentrations[0][:, i]-deviation[:, i], concentrations[0][:, i]+deviation[:, i], color=colors[i], alpha=.2)
    ax1.plot(t, concentrations_ode[o], '--', color=colors[i], alpha=1.0)
ax1.set_ylabel("$\langle x_i(t) \\rangle$")
ax1.set_ylim([0.0, 20.0])

observables = ["x3", "x4", "x5"]
observables_alt = ["$S_2$", "$S_3$", "$S_4$"]
for idx_o, (o, o_alt) in enumerate(zip(observables, observables_alt)):
    i = idx_o + 2
    ax2.plot(t, concentrations[0][:, i], '-', label=o_alt, color=colors[i], alpha=0.7)
    ax2.fill_between(t, concentrations[0][:, i]-deviation[:, i], concentrations[0][:, i]+deviation[:, i], color=colors[i], alpha=.2)
    ax2.plot(t, concentrations_ode[o], '--', color=colors[i], alpha=1.0)
ax2.set_ylabel("$\langle x_i(t) \\rangle$")
ax2.set_ylim([0, 2.5])
ax2.yaxis.tick_right()
ax2.yaxis.set_ticks_position("both")
ax2.yaxis.set_label_position("right")
plt.setp((ax1, ax2), xlabel="$t$", xlim=[0.0, 10.0], xticks=[0.0, 2.5, 5.0, 7.5, 10.0]);

lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(ll, []) for ll in zip(*lines_labels)]
fig.legend(lines, labels, ncols=5, loc="upper center")

plt.savefig("plots/concentrations.pdf");

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), layout='constrained')
for i, o in enumerate(model.observables):
    ax.plot(t, np.abs((concentrations[0][:, i]-concentrations_ode[o.name])), '-', label="$S_{{{}}}$".format(i))
ax.set_xlabel("$t$")
ax.set_ylabel("$|c_i^\mathrm{{CME}}(t) - c_i^\mathrm{{ODE}}(t)|$");
fig.legend(*ax.get_legend_handles_labels(), loc="outside center right")

plt.savefig("plots/concentrations_err.pdf");

## Error depending on rank and partition

### Load data

#### Partition 0

In [ ]:
sliced_err00, marginal_err00 = calculateDistributionError("output/lambda_phage_p0_r5-5-5_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

sliced_err10, marginal_err10 = calculateDistributionError("output/lambda_phage_p0_r5-10-10_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

sliced_err20, marginal_err20 = calculateDistributionError("output/lambda_phage_p0_r5-15-15_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

sliced_err30, marginal_err30 = calculateDistributionError("output/lambda_phage_p0_r5-20-20_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

#### Partition 1

In [ ]:
sliced_err01, marginal_err01 = calculateDistributionError("output/lambda_phage_p1_r5-5-5_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

sliced_err11, marginal_err11 = calculateDistributionError("output/lambda_phage_p1_r5-10-10_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

sliced_err21, marginal_err21 = calculateDistributionError("output/lambda_phage_p1_r5-15-15_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

sliced_err31, marginal_err31 = calculateDistributionError("output/lambda_phage_p1_r5-20-20_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

#### Partition 2

In [ ]:
sliced_err02, marginal_err02 = calculateDistributionError("output/lambda_phage_p2_r5-5-5_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

sliced_err12, marginal_err12 = calculateDistributionError("output/lambda_phage_p2_r5-10-10_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

sliced_err22, marginal_err22 = calculateDistributionError("output/lambda_phage_p2_r5-15-15_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

sliced_err32, marginal_err32 = calculateDistributionError("output/lambda_phage_p2_r5-20-20_i_tau2e-2/output_t7250.nc", P_slice_full, P_sum_full, slice_vec)

#### Inset

In [ ]:
sliced_err_r_5_15_25, marginal_err_r_5_15_25 = calculateDistributionError("output/lambda_phage_p1_r10-15-25_i_tau1e-2/output_t14500.nc", P_slice_matrix, P_sum_matrix, slice_vec)

### Plot

In [ ]:
# from mpl_toolkits.axes_grid1.inset_locator import inset_axes
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
cols = ["$\mathcal{{P}}_0$", "$\mathcal{{P}}_1$", "$\mathcal{{P}}_2$"]
rows = ["$r = (5, 5, 5)$", "$r = (5, 10, 10)$", "$r = (5, 15, 15)$", "$r = (5, 20, 20)$"]
label = "$\max_{{x_i}}|P_M(x_i)-P_M(x_i)^\mathrm{{TTN,ref}}|$"

fig, axs = plt.subplots(4, 3, figsize=(10, 8), sharex='col', sharey='row')

line, = axs[0, 0].plot(np.arange(tree.grid.d()), marginal_err00, '.-', color=colors[0])
axs[1, 0].plot(np.arange(tree.grid.d()), marginal_err10, '.-', color=colors[0])
axs[2, 0].plot(np.arange(tree.grid.d()), marginal_err20, '.-', color=colors[0])
axs[3, 0].plot(np.arange(tree.grid.d()), marginal_err30, '.-', color=colors[0])

axs[0, 1].plot(np.arange(tree.grid.d()), marginal_err01, '.-', color=colors[1])
axs[1, 1].plot(np.arange(tree.grid.d()), marginal_err11, '.-', color=colors[1])
axs[2, 1].plot(np.arange(tree.grid.d()), marginal_err21, '.-', color=colors[1])
axs[3, 1].plot(np.arange(tree.grid.d()), marginal_err31, '.-', color=colors[1])

axs[0, 2].plot(np.arange(tree.grid.d()), marginal_err02, '.-', color=colors[2])
axs[1, 2].plot(np.arange(tree.grid.d()), marginal_err12, '.-', color=colors[2])
axs[2, 2].plot(np.arange(tree.grid.d()), marginal_err22, '.-', color=colors[2])
axs[3, 2].plot(np.arange(tree.grid.d()), marginal_err32, '.-', color=colors[2])

# axins = inset_axes(axs[-1, 1], width='65%', height='65%', loc='upper center')
# axins.plot(np.arange(tree.grid.d()), SSA_marginal_err_r_5_15_25, '--')
# axins.plot(np.arange(tree.grid.d()), DLR_marginal_err_r_5_15_25, '.-')
# axins.set_ylim([-0.02*0.6, 0.42*0.6])
# axins.annotate("TTN with 2 partitions\n($r=5$)", xy=(0.5, 1), xytext=(0, -30),
#                 xycoords='axes fraction', textcoords='offset points',
#                 size=11, ha='center', va='baseline')

plt.setp(axs, ylim=[-0.02, 0.22])
# plt.setp(axs[:, 0], ylim=[-0.02, 0.22])
# plt.setp(axs[:, 1], ylim=[-0.02, 0.47])
# plt.setp(axs[:, 2], ylim=[-0.02, 0.34])

pad = 5 # in points

for ax, col in zip(axs[0], cols):
    ax.annotate(col, xy=(0.5, 1), xytext=(0, pad+5),
                xycoords='axes fraction', textcoords='offset points',
                size=14, ha='center', va='baseline')

for ax, row in zip(axs[:,0], rows):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad-pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size=14, ha='right', va='center')

plt.setp(axs[-1, :], xlabel="species $S_i$", xticks=[0, 2, 4, 6, 8, 10])
fig.suptitle(label, fontsize=16)
plt.subplots_adjust(hspace=.0, wspace=.0)

plt.savefig("plots/comparison_marginal_ttn.pdf", bbox_inches="tight");

## Mass error

### Load data

#### Partition 0

In [ ]:
time_series = TimeSeries("output/lambda_phage_p0_r5-3_i_tau2e-2")
mass_err00 = np.abs(time_series.getMassErr())
time = time_series.time

time_series = TimeSeries("output/lambda_phage_p0_r5-4_i_tau2e-2")
mass_err10 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p0_r5-5_i_tau2e-2")
mass_err20 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p0_r5-10_i_tau2e-2")
mass_err30 = np.abs(time_series.getMassErr())

#### Partition 1

In [ ]:
time_series = TimeSeries("output/lambda_phage_p1_r5-3_i_tau2e-2")
mass_err01 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p1_r5-4_i_tau2e-2")
mass_err11 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p1_r5-5_i_tau2e-2")
mass_err21 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p1_r5-10_i_tau2e-2")
mass_err31 = np.abs(time_series.getMassErr())

#### Partition 2

In [ ]:
time_series = TimeSeries("output/lambda_phage_p2_r5-3_i_tau2e-2")
mass_err02 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p2_r5-4_i_tau2e-2")
mass_err12 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p2_r5-5_i_tau2e-2")
mass_err22 = np.abs(time_series.getMassErr())

time_series = TimeSeries("output/lambda_phage_p2_r5-10_i_tau2e-2")
mass_err32 = np.abs(time_series.getMassErr())

### Plot

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(7, 5))

axs[0, 0].plot(time[1:], mass_err00[1:], label="$\mathcal{{P}}_0$")
axs[0, 0].plot(time[1:], mass_err01[1:], label="$\mathcal{{P}}_1$")
axs[0, 0].plot(time[1:], mass_err02[1:], label="$\mathcal{{P}}_2$")
axs[0, 0].set_title("$r = (5, 3)$")

axs[0, 1].plot(time[1:], mass_err10[1:])
axs[0, 1].plot(time[1:], mass_err11[1:])
axs[0, 1].plot(time[1:], mass_err12[1:])
axs[0, 1].set_title("$r = (5, 4)$")
axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")

axs[1, 0].plot(time[1:], mass_err20[1:])
axs[1, 0].plot(time[1:], mass_err21[1:])
axs[1, 0].plot(time[1:], mass_err22[1:])
axs[1, 0].set_title("$r = (5, 5)$")

axs[1, 1].plot(time[1:], mass_err30[1:])
axs[1, 1].plot(time[1:], mass_err31[1:])
axs[1, 1].plot(time[1:], mass_err32[1:])
axs[1, 1].set_title("$r = (5, 10)$")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")

plt.setp(axs, xlabel="$t$", ylabel="$\Delta m(t)$", xticks=[1.0, 4.0, 7.0, 10.0], ylim=[5e-7, 5e-4], yscale="log")
plt.subplots_adjust(hspace=0.5)
fig.legend(*axs[0, 0].get_legend_handles_labels(), ncols=3, loc="upper center")

plt.savefig("plots/mass_err_comparison_ttn.pdf");

In [ ]:
fig, ax = plt.subplots()
ax.plot(time[1:], mass_err01[1:], label="$r = (5, 3)$")
ax.plot(time[1:], mass_err11[1:], label="$r = (5, 4)$")
ax.plot(time[1:], mass_err21[1:], label="$r = (5, 5)$")
ax.plot(time[1:], mass_err31[1:], label="$r = (5, 10)$")
ax.legend()
ax.ticklabel_format(style='sci', axis='y', scilimits=(-2,2))
plt.setp(ax, xlabel="$t$", ylabel="$\Delta m(t)$", xticks=[1.0, 4.0, 7.0, 10.0])
plt.savefig("plots/mass_err_comparison_ttn_rank.pdf");

## Wall time

### Load data

#### Partition 0

In [ ]:
time_series = TimeSeries("output/lambda_phage_p0_r5-3_i_tau2e-2")
walltime00 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p0_r5-4_i_tau2e-2")
walltime10 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p0_r5-5_i_tau2e-2")
walltime20 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p0_r5-10_i_tau2e-2")
walltime30 = time_series.getWallTime()

#### Partition 1

In [ ]:
time_series = TimeSeries("output/lambda_phage_p1_r5-3_i_tau2e-2")
walltime01 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p1_r5-4_i_tau2e-2")
walltime11 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p1_r5-5_i_tau2e-2")
walltime21 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p1_r5-10_i_tau2e-2")
walltime31 = time_series.getWallTime()

#### Partition 2

In [ ]:
time_series = TimeSeries("output/lambda_phage_p2_r5-3_i_tau2e-2")
walltime02 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p2_r5-4_i_tau2e-2")
walltime12 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p2_r5-5_i_tau2e-2")
walltime22 = time_series.getWallTime()

time_series = TimeSeries("output/lambda_phage_p2_r5-10_i_tau2e-2")
walltime32 = time_series.getWallTime()

### Plot

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(7, 5))
labels = ["$\mathcal{{P}}_{}$".format(i) for i in range(3)]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
walltime0 = [walltime00, walltime01, walltime02]
axs[0, 0].bar(labels, walltime0, color=colors)
axs[0, 0].set_title("$r = (5, 3)$")

walltime1 = [walltime10, walltime11, walltime12]
axs[0, 1].bar(labels, walltime1, color=colors)
axs[0, 1].set_title("$r = (5, 4)$")
axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")

walltime2 = [walltime20, walltime21, walltime22]
axs[1, 0].bar(labels, walltime2, color=colors)
axs[1, 0].set_title("$r = (5, 5)$")

walltime3 = [walltime30, walltime31, walltime32]
axs[1, 1].bar(labels, walltime3, color=colors)
axs[1, 1].set_title("$r = (5, 10)$")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")

plt.setp(axs[:, 0], ylabel="wall time [$\mathrm{s}$]")
# plt.setp(axs, ylim=[1e0, 1e6], yscale="log")
plt.subplots_adjust(hspace=0.3)

plt.savefig("plots/wall_time_comparison_ttn.pdf");

## Memory requirements

In [ ]:
def memoryRequirementsP0(rank):
    x00 = rank[1] * 656
    x01 = rank[1] * 121
    x1 = rank[0] * 11
    q = 1 * rank[0] ** 2
    q0 = rank[0] * rank[1] ** 2
    return (x00 + x01 + x1 + q + q0) * 8 * 1e-3

def memoryRequirementsP1(rank):
    x0 = rank[0] * 656
    x10 = rank[1] * 121
    x11 = rank[1] * 11
    q = 1 * rank[0] ** 2
    q1 = rank[0] * rank[1] ** 2
    return (x0 + x10 + x11 + q + q1) * 8 * 1e-3

def memoryRequirementsP2(rank):
    x00 = rank[1] * 656
    x01 = rank[1] * 11
    x1 = rank[0] * 121
    q = 1 * rank[0] ** 2
    q0 = rank[0] * rank[1] ** 2
    return (x00 + x01 + x1 + q + q0) * 8 * 1e-3

def memoryRequirementsPfull(rank):
    x0 = rank[0] * 656
    x1 = rank[0] * 1331
    q = 1 * rank[0] ** 2
    return (x0 + x1 + q) * 8 * 1e-3

In [ ]:
rank = [5, 3]
fig, axs = plt.subplots(2, 2, figsize=(7, 5))
labels = ["$\mathcal{{P}}_{}$".format(i) for i in range(3)]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
memory_req = [memoryRequirementsP0(rank), memoryRequirementsP1(rank), memoryRequirementsP2(rank)]
axs[0, 0].bar(labels, memory_req, color=colors)
axs[0, 0].set_title("$r = (5, 3)$")

rank = [5, 4]
memory_req = [memoryRequirementsP0(rank), memoryRequirementsP1(rank), memoryRequirementsP2(rank)]
axs[0, 1].bar(labels, memory_req, color=colors)
axs[0, 1].set_title("$r = (5, 4)$")
axs[0, 1].yaxis.tick_right()
axs[0, 1].yaxis.set_ticks_position("both")
axs[0, 1].yaxis.set_label_position("right")

rank = [5, 5]
memory_req = [memoryRequirementsP0(rank), memoryRequirementsP1(rank), memoryRequirementsP2(rank)]
axs[1, 0].bar(labels, memory_req, color=colors)
axs[1, 0].set_title("$r = (5, 5)$")

rank = [5, 10]
memory_req = [memoryRequirementsP0(rank), memoryRequirementsP1(rank), memoryRequirementsP2(rank)]
axs[1, 1].bar(labels, memory_req, color=colors)
axs[1, 1].set_title("$r = (5, 10)$")
axs[1, 1].yaxis.tick_right()
axs[1, 1].yaxis.set_ticks_position("both")
axs[1, 1].yaxis.set_label_position("right")

plt.setp(axs, ylabel="memory [$\mathrm{kB}$]")
# plt.setp(axs, ylim=[1e0, 1e6], yscale="log")
plt.subplots_adjust(hspace=0.3)

plt.savefig("plots/memory_comparison_ttn.pdf");